<a href="https://colab.research.google.com/github/YokoSSS/ProgrammingPython/blob/%E6%95%B0%E7%90%86%E6%9C%80%E9%81%A9%E5%8C%96%E6%BC%94%E7%BF%92(20)/%E6%95%B0%E7%90%86%E6%9C%80%E9%81%A9%E5%8C%96%E6%BC%94%E7%BF%92%EF%BC%8820%EF%BC%892024%E5%89%8D%E6%9C%9F2_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PuLPモジュールを用いた生産計画問題の最適解の計算
## PuLPモジュール読み込み
モジュールの読み込みは
    
    import モジュール名
により行う．

In [ ]:
!pip install matplotlib-venn
!apt-get -qq install -y libfluidsynth1
# https://pypi.python.org/pypi/pydot
!apt-get -qq install -y graphviz && pip install pydot
import pydot
!pip install cartopy
import cartopy
# Install pulp package
!pip install pulp
import pulp


E: Package 'libfluidsynth1' has no installation candidate
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 39.3 MB/s eta 0:00:00


## 決定変数の定義
決定変数のオブジェクト生成する関数は以下の通り．
    
    pulp.LpVariable(name=None,lowBound=None,upBound=None,cat=pulp.LpContinuous)
nameは決定変数の名前で省略不可．nameは他の変数と異なるものにする必要がある．
lowBountは変数の下限値，upBoundは変数の上限値で必要があれば指定する．
catは変数が連続値(実数値）をとる(pulp.LpContinuous)か，整数値をとる(pulp.LpInteger)か，0/1の2値をとる(pulp.LpBinary)かを指定する．catのデフォルトはpulp.LpContinuous．

In [ ]:
x1 = pulp.LpVariable('x1', 0) # x1 >= 0
x2 = pulp.LpVariable('x2', 0) # x2 >= 0

    x1 = pulp.LpVariable('x1', 0)
name（変数名）がx1で下限値が0の実数値をとる決定変数のオブジェクト（インスタンス）x1を生成している．オブジェクトの名前と変数のnameは同じである必要はない．例えば，
    
    x1 = pulp.LpVariable('P1', 0)
としてもよい．変数のnameは他の変数と異なるようにする必要がある．例えば，
    
    x1 = pulp.LpVariable('x', 0)
    x2 = pulp.LpVariable('x', 0)
とすることは許されない．

## 問題の定義
    pulp.LpProblem(name=None, sense=pulp.LpMinimize)
nameは問題名で省略可能．senseは目的で，目的関数を最小化する(pulp.LpMinimize)か，最大化する(pulp.LpMaximize)か指定する．senseのデフォルトはpulp.LpMinimize.

In [ ]:
p = pulp.LpProblem('生産計画問題', sense=pulp.LpMaximize) # 目的関数を最大化
p += x1 + 2*x2, '目的関数　利益見込み'
p += x1 + 3*x2 <= 24, '原料制約'
p += 4*x1 + 4*x2 <= 48, '労働時間制約'
p += 2*x1 + x2 <= 22, '機械稼働制約'
p

生産計画問題:
MAXIMIZE
1*x1 + 2*x2 + 0
SUBJECT TO
原料制約: x1 + 3 x2 <= 24

労働時間制約: 4 x1 + 4 x2 <= 48

機械稼働制約: 2 x1 + x2 <= 22

VARIABLES
x1 Continuous
x2 Continuous

以下の構文で，生産計画問題という名前の，目的が最大化の問題のオブジェクトpを生成している

    p = pulp.LpProblem('生産計画問題', sense=pulp.LpMaximize)
．生成した問題のオブジェクトpに目的関数と制約条件を設定する．目的関数の設定は，
    
    p += x1 + 2*x2, '目的関数　利益見込み'
により行う．右辺に目的関数の式と目的関数の名前を指定する．名前は省略可能．制約条件の設定は以下のように行う．
    
    p += x1 + 3*x2 <= 24, '原料制約'
右辺に制約条件の式と制約条件の名前を指定する．制約条件の不等号および等号は，<=, >=, == を用いる．名前は省略可能．名前をつける場合は重複しないようにする必要がある．

設定した問題の内容は次のように確認できる．

## 最適解の計算と結果の読み取り

In [ ]:
# 最適解を計算
result = p.solve()

In [ ]:
# 最適解が求められたか
pulp.LpStatus[result]

'Optimal'

Optimalなら最適解が得られている．Infeasible（実行可能領域が空），Unbounded（目的関数をどこまでも良くできる）等の場合は最適解が得られていない．

In [ ]:
# 目的関数の最適値（生産計画問題の場合は利益の最大値）
pulp.value(p.objective)

18.0

利益の最大値は18万円

In [ ]:
# 最適解
for v in p.variables():
    print(f'{v} = {pulp.value(v)}')

x1 = 6.0
x2 = 6.0


最適解は x1 = 6, x2 = 6，すなわちP1を6 kg，P2を6 kg生産するときに利益は最大になる．

p.variables()は問題pにおける変数のリストを返す．この例では[x1,x2]を返す．for文は，
    
    for 変数 in 対象:
        処理
で，対象の要素を1つずつ変数に代入して処理を繰り返す．
    
    for v in p.variables():
        print(f'{v} = {pulp.value(v)}')
では，x1, x2について変数名とその値を表示する．print()関数は表示を行う関数である．f''はf-stringと呼ばれる記法で，''内で{}で囲まれた変数や関数等は展開し，他の文字列はそのままを返す．pulp.value(v)は変数vの値を返す．上の例では{v}で変数名が展開され，' = 'はそのまま表示され，{pulp.value(v)}は変数の値が展開される．